<a href="https://colab.research.google.com/github/manavnair2000/BankingApp/blob/master/Plant_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
AUTOTUNE = 1
# import IPython.display as display
# from PIL import Image
import tensorflow_datasets as tfds
import numpy as np
import datetime
import matplotlib.pyplot as plt
# import os
EPOCHS = 35

In [4]:
AUTOTUNE

1

In [0]:
log_dir= "/content/drive/My Drive/Colab Notebooks/src/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [6]:
(ds_train , ds_test, ds_eval), ds_info = tfds.load('plant_village', split=['train[:70%]', 'train[-20%:90%]','train[-10%:]'],as_supervised=True, shuffle_files=True ,with_info=True)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/plant_village/1.0.0.incompleteGZNRO1/plant_village-train.tfrecord


Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.0. Subsequent calls will reuse this data.


In [7]:
print(ds_info.splits['train[:75%]'].num_examples)

40727


In [8]:
tf.data.DatasetSpec.from_value(ds_train)


DatasetSpec((TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorShape([]))

In [0]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
def augment(image,label):
  image = tf.image.resize_with_crop_or_pad(image, 262, 262) # Add 6 pixels of padding
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_saturation(image,1,3)
  image = tf.image.random_contrast(image, 0.2, 0.5)
  image = tf.image.random_crop(image, size=[256, 256, 3]) # Random crop back to 256x256
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
  return image,label
 
ds_train = ds_train.map(normalize_img, num_parallel_calls=4)
ds_train = ds_train.concatenate(ds_train.map(augment, num_parallel_calls=4))
#ds_train = ds_train.cache()
ds_train = ds_train.shuffle(1000)#ds_info.splits['train[:75%]'].num_examples)
ds_train = ds_train.batch(86,drop_remainder = True)
#ds_train = ds_train.prefetch(AUTOTUNE)

In [10]:
tf.data.DatasetSpec.from_value(ds_train)


DatasetSpec((TensorSpec(shape=(86, None, None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(86,), dtype=tf.int64, name=None)), TensorShape([]))

In [0]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=4)
# ds_test = ds_test.shuffle(ds_info.splits['train[-25%:]'].num_examples)
ds_test = ds_test.batch(160,drop_remainder = True)
# ds_test = ds_test.cache()
# ds_test = ds_test.prefetch(AUTOTUNE)

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(20,5,strides=(1,1),input_shape=(256,256,3),padding='same',kernel_initializer='VarianceScaling',activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.23),
    tf.keras.layers.Conv2D(27,5,strides=(1,1),padding='same',kernel_initializer='VarianceScaling',activation='relu'), #input_shape=(128,128,3),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.23),
    tf.keras.layers.Conv2D(36,5,strides=(1,1),padding='same',kernel_initializer='VarianceScaling',activation='relu'), #input_shape=(64,64,3),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.23),
    tf.keras.layers.Conv2D(48,5,strides=(1,1),padding='same',kernel_initializer='VarianceScaling',activation='relu'), #input_shape=(28,28,3),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.23),
    tf.keras.layers.Conv2D(64,5,strides=(1,1),padding='same',kernel_initializer='VarianceScaling',activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same'),
    tf.keras.layers.Dropout(0.23),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16281,activation='relu',kernel_initializer='VarianceScaling'),
    tf.keras.layers.Dense(38,activation='softmax',kernel_initializer='VarianceScaling')
])

In [0]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0095,momentum=0.070),
    metrics=['accuracy']
)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 20)      1520      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 20)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 20)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 27)      13527     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 27)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 27)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 36)        2

In [15]:
history = model.fit( ds_train,
    epochs=EPOCHS,
    verbose=1,
    validation_data=ds_test,
    callbacks=[callbacks]
) 

Epoch 1/35
884/884 [==============================] - 260s 294ms/step - loss: 2.8654 - accuracy: 0.2422 - val_loss: 2.4195 - val_accuracy: 0.2930
Epoch 2/35
884/884 [==============================] - 269s 305ms/step - loss: 1.5627 - accuracy: 0.5441 - val_loss: 1.4200 - val_accuracy: 0.5746
Epoch 3/35
884/884 [==============================] - 271s 306ms/step - loss: 1.0695 - accuracy: 0.6775 - val_loss: 1.0726 - val_accuracy: 0.6706
Epoch 4/35
884/884 [==============================] - 274s 310ms/step - loss: 0.8086 - accuracy: 0.7542 - val_loss: 1.0650 - val_accuracy: 0.6845
Epoch 5/35
884/884 [==============================] - 271s 307ms/step - loss: 0.6556 - accuracy: 0.7988 - val_loss: 0.8392 - val_accuracy: 0.7473
Epoch 6/35
884/884 [==============================] - 270s 306ms/step - loss: 0.5517 - accuracy: 0.8303 - val_loss: 0.7967 - val_accuracy: 0.7667
Epoch 7/35
884/884 [==============================] - 268s 303ms/step - loss: 0.4649 - accuracy: 0.8543 - val_loss: 0.8172 -

In [0]:

# ds_eval = ds_eval.shuffle(1000)
# ds_eval = ds_eval.batch(64,drop_remainder = True)
# model.evaluate(ds_eval,verbose=1)

In [0]:
ds_info.splits['train[-10%:]'].num_examples

In [0]:
%tensorflow -V

In [16]:
for image,label in ds_eval.take(39):
  print("Actual output label:",ds_info.features["label"].names[label])
  #print(tf.cast(image, tf.float32) / 255.)
  imageT = tf.cast(image, tf.float32) / 255.
  imageT = tf.cast(imageT, tf.float32)
  #imageT = tf.reshape(imageT,(-1,imageT.get_shape()))
  imageT = tf.reshape(imageT,(-1,256,256,3))
  #print(imageT.get_shape())
  #print((tf.cast(image, tf.float32) / 255.).set_shape((256,256,3,)))
  print("Predicted output label:",ds_info.features["label"].names[model.predict_classes(imageT,batch_size=1)[0]])

Actual output label: Apple___Black_rot
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Predicted output label: Apple___Black_rot
Actual output label: Potato___Late_blight
Predicted output label: Potato___Late_blight
Actual output label: Corn___healthy
Predicted output label: Corn___healthy
Actual output label: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Predicted output label: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Actual output label: Tomato___Tomato_mosaic_virus
Predicted output label: Tomato___Tomato_mosaic_virus
Actual output label: Orange___Haunglongbing_(Citrus_greening)
Predicted output label: Orange___Haunglongbing_(Citrus_greening)
Actual output label: Peach___Bacterial_spot
Predicted output label: Peach___Bacterial_spot
Actual output label: Pepper,_bell___healthy
Predicted output label: Pepper,_bell___healthy
Actual output label: Peach___Bacterial_spot
Predicted output label: Peach___Bacterial_spot
Actual output label: Potato___Early_blight
Predicted output label: Potato___Early_blight
Actual output label: Orange___Haunglongbing_(Citrus_greening)
Predicted output la